### Local Model Definition
The create_local_model() function defines the architecture of the lightweight neural network used for both local training and the global model. The model consists of two hidden dense layers with ReLU activation functions and dropout for regularization, followed by a sigmoid output layer suitable for binary classification. This standardized structure ensures consistency across all client models, which is crucial for weight aggregation.

In [ ]:
# Local Model Definition
def create_local_model(input_dim=2):
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(input_dim,)),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    return model


### 2. Local Model Training
During each training round, a new local model is instantiated per device and initialized with the current global model weights. The model is then trained on locally available data using a differentially private optimizer (DPKerasSGDOptimizer) to ensure that each update preserves user privacy. Training is conducted for one epoch per device to simulate a real-world on-device federated learning scenario. After training, local weights, loss, and accuracy are recorded for each client.

In [ ]:
# Training Round
for i, client_dataset in enumerate(client_data):
    local_model = create_local_model()
    local_model.set_weights(global_model.get_weights())
    local_optimizer = tfp.DPKerasSGDOptimizer(
        l2_norm_clip=1.0,
        noise_multiplier=noise_multiplier,
        num_microbatches=1,
        learning_rate=0.01
    )
    local_model.compile(
        optimizer=local_optimizer,
        loss='binary_crossentropy',
        metrics=['binary_accuracy']
    )
    logging.info(f"Training local model on device {i + 1}")
    history = local_model.fit(client_dataset, epochs=1, verbose=0)
    
    local_weights.append(local_model.get_weights())
    local_losses.append(history.history['loss'][0])
    local_accs.append(history.history['binary_accuracy'][0])


### 3. Model Aggregation (Federated Averaging)
Once all local models are trained, their weights are aggregated using the Federated Averaging (FedAvg) algorithm. This method computes the element-wise mean of the model weights from each device to form an updated global model. This central model is then redistributed to clients in the next round. FedAvg is a key step in federated learning, allowing global updates without centralized data collection.

In [ ]:
def aggregate_weights(local_weights):
    return [np.mean([w[i] for w in local_weights], axis=0) for i in range(len(local_weights[0]))]

global_weights = aggregate_weights(local_weights)
global_model.set_weights(global_weights)


### 4. Central Model Initialization and Synchronization
At the beginning of training, a central global model is initialized with the same architecture as the local models. It acts as the synchronized model shared across all clients and is continuously updated after each round of aggregation. The synchronization process ensures that every client trains from the same starting point in every round, maintaining consistency across decentralized learning processes.

In [ ]:
global_model = create_local_model()
optimizer = tfp.DPKerasSGDOptimizer(
    l2_norm_clip=1.0,
    noise_multiplier=noise_multiplier,
    num_microbatches=1,
    learning_rate=0.01
)
global_model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['binary_accuracy']
)


### 5. Privacy and Performance Metrics
After each round, key performance and privacy metrics are computed. Mutual information is used to estimate potential privacy leakage from model updates. The dp_accounting library computes the differential privacy budget (ε) using Rényi Differential Privacy accounting. Additional metrics such as latency, global accuracy, and loss are tracked to monitor model convergence and privacy trade-offs across training rounds.

In [ ]:
leak = compute_mutual_information(raw_data[0][0], updates)
lat = np.random.uniform(0.1, 0.5) * num_devices

accountant = dp_accounting.rdp.RdpAccountant()
steps = (total_examples // batch_size) * (round_num + 1)
event = dp_accounting.GaussianDpEvent(noise_multiplier=noise_multiplier)
accountant.compose(dp_accounting.SelfComposedDpEvent(event, steps))
epsilon = accountant.get_epsilon(target_delta=1e-5)


###  6. Metrics Logging and Model Saving
All performance metrics, including accuracy, loss, leakage, latency, and privacy budget (ε), are saved for analysis. The final global model weights are saved locally for reproducibility or deployment. This step ensures traceability and provides insights into the model’s privacy-utility trade-offs over multiple federated learning rounds.

In [ ]:
os.makedirs('ftl_app/precomputed/weights', exist_ok=True)
global_model.save_weights('ftl_app/precomputed/weights/global_model.h5')

with open('ftl_app/precomputed/metrics.json', 'w') as f:
    json.dump({
        'accuracy': accuracy, 'loss': loss,
        'leakage': leakage, 'latency': latency, 'epsilon': epsilon_values
    }, f)


### Full MOdel Pipelein

In [ ]:
from ftl_app.ftl_core.ftl_models import run_ftl_simulation
import json

accuracy, loss, leakage, latency, epsilon = run_ftl_simulation(num_rounds=10, num_devices=10, noise_multiplier=1.1)
with open('ftl_app/precomputed/metrics.json', 'w') as f:
    json.dump({
        'accuracy': accuracy, 'loss': loss, 'leakage': leakage, 
        'latency': latency, 'epsilon': epsilon
    }, f)
print("Precomputation completed successfully!")